# Smart-G demo notebook for Objects

This is an interactive document allowing to run Smart-G with python and visualize the results. <br>
*Tips*: cells can be executed with shift-enter. Tooltips can be obtained with shift-tab. More information [here](http://ipython.org/notebook.html) or in the help menu. [A table of content can also be added](https://github.com/minrk/ipython_extensions#table-of-contents).

In [ ]:
%pylab inline
# next 2 lines allow to automatically reload modules that have been changed externally
%reload_ext autoreload
%autoreload 2

import os, sys
sys.path.insert(0, os.path.dirname(os.getcwd()))
from smartg.smartg import Smartg
from smartg.smartg import LambSurface
from smartg.atmosphere import AtmAFGL, AeroOPAC

import smartg.geometry
from smartg.geometry import Point
import smartg.visualizegeo
from smartg.visualizegeo import receiver_view, cat_view, Mirror, Plane, Transformation, \
    Entity, Analyse_create_entity, Matte
from smartg.smartg import CusForward

# Quick Start

## Creation of objects

In [ ]:
# We want to create a simple case with a receiver and four heliostats (MirA, B, C and D).
# The receiver will be created a x = 1km, the first heliostat at x = 950m, the
# second heliostat at x = 900m and so on. The mirror will be oriented such that the
# direct rays reflected by the mirrors will totally reach the receiver.

# The solar zenith angle
solarDir = 14.3
# Create the heliostats MirA, MirB, MirC and MirD (smartg unit is the kilometer)
wMx = 0.004725
wMy = 0.00642
MirA = Entity(name = "reflector", \
              materialAV = Mirror(reflectivity = 0.88), \
              materialAR = Matte(reflectivity = 0.), \
              geo = Plane( p1 = Point(-wMx, -wMy, 0.),
                           p2 = Point(wMx, -wMy, 0.),
                           p3 = Point(-wMx, wMy, 0.),
                           p4 = Point(wMx, wMy, 0.) ), \
              transformation = Transformation( rotation = np.array([0., 20.281725, 0.]), \
                                               translation = np.array([0.95, 0., 0.00517]) ))

MirB = Entity(MirA); MirC = Entity(MirA); MirD = Entity(MirA);
MirB.transformation = Transformation( rotation = np.array([0., 29.460753, 0.]), \
                                      translation = np.array([0.9, 0., 0.00517]) )
MirC.transformation = Transformation( rotation = np.array([0., 35.129831, 0.]), \
                                      translation = np.array([0.85, 0., 0.00517]) )
MirD.transformation = Transformation( rotation = np.array([0., 38.715473, 0.]), \
                                      translation = np.array([0.8, 0., 0.00517]) )

# Create the receiver Recept1
wRx = 0.006
wRy = 0.007
# TC = Taille cellule. The receiver is devided in several cells to visualize the flux distribution 
Recept1 = Entity(name = "receiver", TC = 0.0005, \
                 materialAV = Matte(reflectivity = 0.), \
                 materialAR = Matte(reflectivity = 0.), \
                 geo = Plane( p1 = Point(-wRx, -wRy, 0.),
                              p2 = Point(wRx, -wRy, 0.),
                              p3 = Point(-wRx, wRy, 0.),
                              p4 = Point(wRx, wRy, 0.) ), \
                 transformation = Transformation( rotation = np.array([0., -101.5, 0.]), \
                                                  translation = np.array([1., 0., 0.1065]) ))

# Create the list containing all the objects
listobjs = [MirA, MirB, MirC, MirD, Recept1]

## Run the simulation

In [ ]:
# In this simulation the photons from TOA are launched to fill only the
# mirrors (ray tracing method or restricted forward method) --> cusForward = CusForward(LMODE="RF")
# By default the sun is a ponctual sun source targeting the origin (0,0,0) --> --> cusForward = None
# For a full forward mode i.g. specify in Smartg.run() -> cusForward = CusForward(CFX=10, CFY=10, LMODE="FF")
# Where CFX is the size in kilometer in the x axis of the rectangle. Be careful, the full forward mode 
# need a big number of photons to obtain a good accuracy !

w2 = 120.
Pmin = [-w2, -w2, -0.5]
Pmax = [w2, w2, 121]
interval0 = [Pmin, Pmax]
aer = AeroOPAC('desert', 0.25, 550.)
pro = AtmAFGL('afglms', comp=[aer], P0 = 877, H2O=1.2)
m = Smartg(double = True, obj3D = True).run( surf = LambSurface(ALB=0.25), 
    THVDEG=solarDir, NF=1e6, wl=550., NBPHOTONS=1e8, NBLOOP = 1e7, atm=pro,
    myObjects=listobjs, interval = interval0, cusForward = CusForward(LMODE="RF"))

## How to show the results ?

In [ ]:
# Show the description of the simulation
m.print_info(show_attrs=True)

In [ ]:
# 3D print of objects, the notebook give only a given 2D print of the 3D one
fig1 = Analyse_create_entity(listobjs, Theta = solarDir)

In [ ]:
# print the infomation at the receiver
cat_view(m)

In [ ]:
# print the flux distribution at the receiver
receiver_view(m['C_Receiver'][:,:], [wRx, wRy], logI=False, nameFile = None)

In [ ]:
# print the flux distribution at the receiver from only the direct (cat1)
receiver_view(m['C1_Receiver'][:,:], [wRx, wRy], logI=False, nameFile = '1H_02_CAT1')

In [ ]:
# print the flux distribution at the receiver from beams from the process H only (cat2)
receiver_view(m['C2_Receiver'][:,:], [wRx, wRy], logI=False, nameFile = '1H_02_CAT2')

In [ ]:
# print the flux distribution at the receiver from beams from the process E only (cat3)
receiver_view(m['C3_Receiver'][:,:], [wRx, wRy], logI=False, nameFile = '1H_02_CAT3')

In [ ]:
# print the flux distribution at the receiver from beams from the process A only (cat4)
receiver_view(m['C4_Receiver'][:,:], [wRx, wRy], logI=False, nameFile = '1H_02_CAT4')

In [ ]:
# print the flux distribution at the receiver from beams from the two processes H and A (cat5)
receiver_view(m['C5_Receiver'][:,:], [wRx, wRy], logI=True, nameFile = '1H_02_CAT5')

In [ ]:
# print the flux distribution at the receiver from beams from the two processes H and E (cat6)
receiver_view(m['C6_Receiver'][:,:], [wRx, wRy], logI=False, nameFile = '1H_02_CAT6')

In [ ]:
# print the flux distribution at the receiver from beams from the two processes E and A (cat7)
receiver_view(m['C7_Receiver'][:,:], [wRx, wRy], logI=False, nameFile = '1H_02_CAT7')

In [ ]:
# print the flux distribution at the receiver from beams from the three processes H , E and A (cat8)
receiver_view(m['C8_Receiver'][:,:], [wRx, wRy], logI=False, nameFile = '1H_02_CAT8')